# EDC Project
## Task 1
Plot confusion matrix and find error rate


In [2]:
!pip install scikit-learn
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from IPython.display import display
from no_pipeline import Dataset

feat = ['spectral_centroid_mean', 'spectral_rolloff_mean', 'mfcc_1_mean', 'tempo']
genre_data = Dataset('Classification music/GenreClassData_30s.txt', 5, feat, None)

genre_data.scale()
error_rate = genre_data.classify(True)
print(error_rate)
plt.show()



AttributeError: type object 'ConfusionMatrixDisplay' has no attribute 'from_estimator'